In [1]:
# Importing all dependent APIs
import os, cv2, random, time
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
print("Importing Dependencies Completed.")

Using TensorFlow backend.


Importing Dependencies Completed.


In [2]:
# Creating a database for running CNN
dataDir = r"E:\DeepLearning\ImageDatasets\Dataset01-EBI-McD180mlBottom\EBIResult\Images"
categories = ["Pass", "Fail"]
trainingData = []
xTrain = []
yTrain = []
xTest = []
yTest = []
imgSizeC = 256

def createTrainingData():
    for category in categories:
        path = os.path.join(dataDir, category)    # path to cats or dogs dir
        print("Loading " + category + " images.")
        time.sleep(0.25)
        classNum = categories.index(category)
        loop = tqdm(total=len(os.listdir(path)),desc=category, position=0, leave=False)
        for img in os.listdir(path):
            try:
                imgArray = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                cropped = imgArray[:, 88:1112]
                newArrC = cv2.resize(cropped, (imgSizeC, imgSizeC))
                trainingData.append([newArrC, classNum])
                loop.update()
            except Exception as e:
                pass
    random.shuffle(trainingData)

createTrainingData()

print("Length of training data: ",len(trainingData))

for featuresTrain, labelTrain in trainingData[:24000]:
    xTrain.append(featuresTrain)
    yTrain.append(labelTrain)
    
for featuresTest, labelTest in trainingData[24000:]:
    xTest.append(featuresTest)
    yTest.append(labelTest)

xTrain = np.array(xTrain).reshape(-1, imgSizeC, imgSizeC, 1)
xTest = np.array(xTest).reshape(-1, imgSizeC, imgSizeC, 1)

print(xTrain.shape)
print(xTest.shape)

#print(yTrain.shape)
#print(yTest.shape)

print("Done")

Loading Pass images.


Pass: 100%|█████████████████████████████████████████████████████████████████████▉| 12815/12821 [03:12<00:00, 70.45it/s]

Loading Fail images.


Fail: 100%|█████████████████████████████████████████████████████████████████████▉| 12818/12821 [03:07<00:00, 69.69it/s]

Length of training data:  25642
(24000, 256, 256, 1)
(1642, 256, 256, 1)
Done


In [3]:
# Pre-processing stage
# Reshaping loaded image dataset to 3 dimensions and plotting sample image.
#print(xTrain.shape) # 200, 256, 256, 1
#xTrain = xTrain.astype('float32')
xTrain = np.array(xTrain, dtype=np.float32)
#xTest = xTest.astype('float32')
xTest = np.array(xTest, dtype=np.float32)
xTrain /= 255
xTest /= 255

Fail: 100%|██████████████████████████████████████████████████████████████████████| 12821/12821 [03:26<00:00, 69.69it/s]

In [20]:
print(len(yTrain))
print(yTrain[:10])
# Train & Test datasets are represented as single array with class values.

# Splitting Train & Test datasets in 2 distinct class labels
# Convert 1-D Class arrays to 2-D Class matrices
yTrain = np_utils.to_categorical(yTrain,2)
yTest = np_utils.to_categorical(yTest,2)
print("========================")
print(yTrain.shape)
print(yTrain[:10])

24000
[1, 0, 1, 0, 0, 0, 1, 0, 1, 1]
(24000, 2)
[[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [21]:
# Declaring sequential model format
model = Sequential()
model.add(Convolution2D(32,(3,3),activation='relu',input_shape=(256,256,1)))
print(model.output_shape)
# (None, 254, 254, 32)

model.add(Convolution2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))

(None, 254, 254, 32)


In [22]:
# Model compilation
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
# Model fitting on Training data
model.fit(xTrain,yTrain,batch_size=32,epochs=10,verbose=1)

Epoch 1/10
24000/24000 [==============================] - 1754s 73ms/step - loss: 0.4258 - acc: 0.8906
Epoch 2/10
24000/24000 [==============================] - 1908s 80ms/step - loss: 0.0818 - acc: 0.9745
Epoch 3/10
24000/24000 [==============================] - 1798s 75ms/step - loss: 0.0400 - acc: 0.9880
Epoch 4/10
24000/24000 [==============================] - 1849s 77ms/step - loss: 0.0296 - acc: 0.9913
Epoch 5/10
24000/24000 [==============================] - 1812s 75ms/step - loss: 0.0202 - acc: 0.9939
Epoch 6/10
24000/24000 [==============================] - 1812s 75ms/step - loss: 0.0177 - acc: 0.9948
Epoch 7/10
24000/24000 [==============================] - 1805s 75ms/step - loss: 0.0180 - acc: 0.9953
Epoch 8/10
24000/24000 [==============================] - 1792s 75ms/step - loss: 0.0151 - acc: 0.9965
Epoch 9/10
24000/24000 [==============================] - 1841s 77ms/step - loss: 0.0095 - acc: 0.9982
Epoch 10/10
24000/24000 [==============================] - 1930s 80ms/ste

In [10]:
# Model evaluation
[loss, accuracy] = model.evaluate(xTest,yTest,verbose=1)
print("Loss: ", loss)
print("Accuracy: ", accuracy*100)

50/50 [==============================] - 1s 15ms/step
Loss:  0.06790319502353669
Accuracy:  97.99999904632568
